In [1]:
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt

import sddk

In [2]:
s = sddk.cloudSession("sciencedata.dk", "SDAM_root", "648597@au.dk")
figpath = "SDAM_data/social_diversity/socdiv_figures/"

# global parameters for plots
plt.style.use("seaborn-white")
plt.rcParams['figure.dpi'] = 300
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams['font.size'] = 8

def save_figure(fig, fig_name):
    #each figure to be saved in four different versions
    fig.savefig("../figures/" + fig_name + ".eps", format="eps")
    fig.savefig("../figures/" + fig_name + ".png")
    s.write_file(figpath + fig_name + ".eps", fig)
    s.write_file(figpath + fig_name + ".png", fig)

connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [3]:
roman_cities = gpd.read_file("../data/roman_cities_pop.geojson")

In [4]:
# Hanson et al. 2017 on associations
hanson_assoc = pd.read_excel("../data/hanson2017_supplementary.xlsx")
hanson_assoc.head(5)

,Primary key,Name,Province,Population,associations,inscriptions,D(N),West / East
0,992,Roma,Italia (VII Etruria),923406,155,119532,1197.402620,West
1,110,Alexandria (Aegyptus),Aegyptus,410535,7,203,14156.379310,East
2,172,Carthago,Africa Proconsularis,102170,7,8445,84.687981,West
3,1320,Syracusae,Silicia,83834,1,578,145.041522,West
4,276,Ephesus,Asia,71587,4,373,767.689008,East


In [5]:
hanson_assoc["ID"] = hanson_assoc["Primary key"].apply(str)

In [6]:
LIRE = gpd.read_parquet("../data/large_data/LIRE_occupsorgs_cats.parquet")

In [7]:
def get_5K_city(row):
    city_id = None
    if row["large_city_dist"] <= 5000:
        city_id = row["large_city_ID"]
    else:
        if row["medium_city_dist"] <= 5000:
            city_id = row["medium_city_ID"]
        else:
            if row["small_city_dist"] <= 5000:
                city_id = "small"
    return city_id

LIRE["5Kcity"] = LIRE.apply(lambda row: get_5K_city(row), axis=1)

In [8]:
def get_inscriptions(ID):
    city_dict = {}
    city_data = LIRE[LIRE["5Kcity"]==str(ID)]
    if len(city_data) > 0:
        city_dict["inscr_N"] = len(city_data)
        city_dict["occups_list"] = [el for sublist in city_data["occups"] for el in sublist]
        city_dict["occups_N"] = city_data["occups_N"].sum()
        city_dict["organizations_N"] = city_data["organizations_N"].sum()
        city_dict["organizations"] = [el for sublist in city_data["organizations"] for el in sublist]
    return city_dict

cities_dict_data = roman_cities["ID"].apply(get_inscriptions)
cities_inscr_data = pd.DataFrame(cities_dict_data.tolist())

In [9]:
roman_cities = pd.concat([roman_cities, cities_inscr_data], axis=1)

In [10]:
cities_merged = pd.merge(roman_cities, hanson_assoc, on="ID", how="inner")
cities_merged.head(5)

,Primary Key,Ancient Toponym,Modern Toponym,Province_x,Country,Barrington Atlas Rank,Barrington Atlas Reference,Start Date,End Date,Select Bibliography,...,organizations_N,organizations,Primary key,Name,Province_y,Population,associations,inscriptions,D(N),West / East
0,Hanson2016_13,Argos (Achaea) (2),Argos,Achaea,Greece,2,58 D2,-900,NaN,BNP; Boatwright 2000; DGRG; Hansen 2006; Hanse...,...,0.0,[],13,Argos,Achaea,49453,1,24,2060.541667,East
1,Hanson2016_14,Athenae,Athens,Achaea,Greece,1,59 B3,-900,NaN,Beloch 1886; BNP; Boatwright 2000; DGRG; Hanse...,...,0.0,[],14,Athenae,Achaea,58114,5,275,1056.618182,East
2,Hanson2016_28,Delos,Delos,Achaea,Greece,4 or 5,61 A3,-700,NaN,BNP; DGRG; Hansen and Nielsen 2004; PECS; Sear...,...,NaN,NaN,28,Delos,Achaea,18364,7,149,862.738255,East
3,Hanson2016_110,Alexandria (Aegyptus),Alexandria,Aegyptus,Egypt,1,74 B2,-322,NaN,Beloch 1886; Bowman 2011; Cohen 2006; Humphrey...,...,0.0,[],110,Alexandria (Aegyptus),Aegyptus,410535,7,203,14156.379310,East
4,Hanson2016_172,Carthago,Tunis,Africa Proconsularis,Tunisia,1,32 F3,-49,NaN,Bomgardner 2000; Brunt 1971; Cornell and Mathe...,...,0.0,[],172,Carthago,Africa Proconsularis,102170,7,8445,84.687981,West


In [11]:
len(cities_merged)

210

In [67]:
cities_merged[['inscr_N', 'occups_N', "organizations_N", "associations", "inscriptions"]]

,inscr_N,occups_N,organizations_N,associations,inscriptions
0,7.0,0.0,0.0,1,24
1,60.0,1.0,0.0,5,275
2,NaN,NaN,NaN,7,149
3,35.0,0.0,0.0,7,203
4,409.0,8.0,0.0,7,8445
...,...,...,...,...,...
205,13.0,0.0,0.0,1,69
206,NaN,NaN,NaN,1,28
207,5.0,0.0,0.0,1,15
208,4.0,0.0,0.0,1,22


In [12]:
len(cities_merged)

210

In [13]:
cities_merged_w = cities_merged[(cities_merged["western_province"]==True) & (cities_merged["Ancient Toponym"]!="Roma")]
len(cities_merged_w)

175

In [16]:
cities_merged_w

,Primary Key,Ancient Toponym,Modern Toponym,Province_x,Country,Barrington Atlas Rank,Barrington Atlas Reference,Start Date,End Date,Select Bibliography,...,organizations_N,organizations,Primary key,Name,Province_y,Population,associations,inscriptions,D(N),West / East
4,Hanson2016_172,Carthago,Tunis,Africa Proconsularis,Tunisia,1,32 F3,-49,NaN,Bomgardner 2000; Brunt 1971; Cornell and Mathe...,...,0.0,[],172,Carthago,Africa Proconsularis,102170,7,8445,84.687981,West
5,Hanson2016_210,Thugga,Dougga,Africa Proconsularis,Tunisia,3,32 D4,-200,NaN,BNP; Lepelley 1979-1981; PECS; Quinn and Wilso...,...,NaN,NaN,210,Thugga,Africa Proconsularis,3085,1,2028,1.521203,West
6,Hanson2016_223,Augusta Taurinorum,Turin,Alpes Cottiae,Italy,3,39 B3,-27,NaN,Beloch 1880; Beloch 1886; BNP; Cornell and Mat...,...,2.0,"[sodalicium, sodalicium]",223,Augusta Taurinorum,Alpes Cottiae,8633,4,352,98.102273,West
7,Hanson2016_224,Segusio,Susa,Alpes Cottiae,Italy,4 or 5,17 I3,-100,NaN,Beloch 1880; de Ligt 2012; DGRG; Golvin 1988; ...,...,NaN,NaN,224,Segusio,Alpes Cottiae,563,1,163,3.453988,West
8,Hanson2016_227,Cemelenum,Cimiez,Alpes Maritimae,France,3,16 D2,-25,NaN,Bedon 2001; Bedon et al 1988; Beloch 1880; de ...,...,NaN,NaN,227,Cemelenum,Alpes Maritimae,1989,5,295,33.711864,West
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,Hanson2016_1288,Augusta Vindelicum,Augsburg,Raetia,Germany,2,12 D4,-15,NaN,BNP; Boatwright 2000; McEvedy 2011; PECS; Quin...,...,0.0,[],1288,Augusta Vindelicum,Raetia,10608,2,317,66.927445,West
199,Hanson2016_1298,Catania,Catania,Sicilia,Italy,3,47 G3,-729,NaN,Benario 1981; Brunt 1971; Cornell and Mathews ...,...,0.0,[],1298,Catania,Silicia,28211,2,658,85.747720,West
200,Hanson2016_1312,Messana,Messina,Sicilia,Italy,3,47 H2,-730,NaN,Brunt 1971; Cornell and Mathews 1982; Hansen 2...,...,0.0,[],1312,Messana,Silicia,11531,1,156,73.916667,West
201,Hanson2016_1316,Panormus,Palermo,Sicilia,Italy,2,47 C2,-700,NaN,Beloch 1886; BNP; Brunt 1971; Cornell and Math...,...,0.0,[],1316,Panormus,Silicia,5067,1,213,23.788732,West


In [14]:
cities_merged_quant = cities_merged[cities_merged["Ancient Toponym"]!="Roma"][['inscr_N', 'occups_N', 'organizations_N', "associations", "inscriptions", "D(N)"]]

In [15]:
corr_matrix = cities_merged_quant.corr().round(2)

In [80]:
corr_matrix

,inscr_N,occups_N,organizations_N,associations,inscriptions,D(N)
inscr_N,1.00,0.69,0.68,0.50,0.76,-0.01
occups_N,0.69,1.00,0.88,0.71,0.47,0.01
organizations_N,0.68,0.88,1.00,0.59,0.40,0.01
associations,0.50,0.71,0.59,1.00,0.60,0.08
inscriptions,0.76,0.47,0.40,0.60,1.00,-0.08
D(N),-0.01,0.01,0.01,0.08,-0.08,1.00
